Implementation of the following paper: https://arxiv.org/pdf/1211.0358.pdf

In [ ]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
%matplotlib inline

## Sampling from deep GP

Considering the case with 1D input layer, 1 2D hidden layer and 2D output layer

Selecting input layer and hyperparameters

In [ ]:
x = np.linspace(0, 1, 100)

sigma_hidden = [1, 2]
l_hidden = [0.25, 0.5]

y_sigma = [1, 2]
y_weights = [[5, 10], [1, 20]]

First generating a hidden layer

In [ ]:
x1_matrix = np.tile(x, len(x)).reshape((len(x), len(x)))
x2_matrix = np.tile(x, len(x)).reshape((len(x), len(x))).transpose()

h0_K = np.exp(-(x1_matrix - x2_matrix) ** 2 / (2 * l_hidden[0] ** 2)) * sigma_hidden[0] ** 2
h1_K = np.exp(-(x1_matrix - x2_matrix) ** 2 / (2 * l_hidden[1] ** 2)) * sigma_hidden[1] ** 2
while True:
    try:
        np.linalg.cholesky(h0_K)
        break
    except:
        h0_K += 0.0001 * np.eye(len(x))
while True:
    try:
        np.linalg.cholesky(h1_K)
        break
    except:
        h1_K += 0.0001 * np.eye(len(x))
        
h0 = np.dot(np.linalg.cholesky(h0_K), np.random.randn(len(x)).reshape(-1, 1))
h1 = np.dot(np.linalg.cholesky(h1_K), np.random.randn(len(x)).reshape(-1, 1))

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(x, h0, label="h0")
plt.plot(x, h1, label="h1")
plt.legend()

plt.subplot(2, 1, 2)
plt.scatter(h0, h1)
plt.show()

Generating an output layer

In [ ]:
h0_matrix = np.tile(h0, len(h0)).reshape((len(h0), len(h0)))
h0_matrixT = h0_matrix.transpose()
h1_matrix = np.tile(h1, len(h1)).reshape((len(h1), len(h1)))
h1_matrixT = h1_matrix.transpose()

y0_K = np.exp(- y_weights[0][0] * (h0_matrix - h0_matrixT) ** 2 / 2 
              - y_weights[0][1] * (h1_matrix - h1_matrixT) ** 2 / 2 ) * y_sigma[0] ** 2
y1_K = np.exp(- y_weights[1][0] * (h0_matrix - h0_matrixT) ** 2 / 2 
              - y_weights[1][1] * (h1_matrix - h1_matrixT) ** 2 / 2 ) * y_sigma[1] ** 2
while True:
    try:
        np.linalg.cholesky(y0_K)
        break
    except:
        y0_K += 0.0001 * np.eye(len(x))
while True:
    try:
        np.linalg.cholesky(y1_K)
        break
    except:
        y1_K += 0.0001 * np.eye(len(x))

y0 = np.dot(np.linalg.cholesky(y0_K), np.random.randn(len(x)).reshape(-1, 1))
y1 = np.dot(np.linalg.cholesky(y1_K), np.random.randn(len(x)).reshape(-1, 1))

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(x, y0, label="y0")
plt.plot(x, y1, label="y1")
plt.legend()

plt.subplot(2, 1, 2)
plt.scatter(y0, y1)
plt.show()

## Learning deep GP process